In [ ]:
! pip install -U accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
! pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#importing common_voice_14_0 dataset which contains "4008" Marathi audio samples and transcription text


from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

# splitting training(4008) and testing data(1737 samples) ) using split

common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_14_0", "mr", split="train+validation"
)
common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_14_0", "mr", split="test"
)

print(common_voice)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_14_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_14_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2225it [00:00, 60672.41it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1783it [00:00, 80623.17it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2826it [00:00, 61958.54it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 2259it [00:00, 63868.34it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_14_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_14_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 4008
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
        num_rows: 1737
    })
})


In [ ]:
# imported dataset contains other columns like age, gender, locale etc which are not required for training,
# so selecting only audio and sentence columns

common_voice = common_voice.select_columns(["audio", "sentence"])

In [ ]:
# Found sampling rate of 48000 Hz for dataset which we will match with pre trained processor in further steps
common_voice["train"].features

{'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [ ]:
#checking whether dataset imported is of Marathi text or not before further proceeding
common_voice["train"][0]['sentence']

'आईचे आजारपण वाढत चालले, तसतशी मथीही नीट खातपीतनाशी झाली.'

In [ ]:
#Importing pre trained whisper processor for feature extraction of data and encoding ,decoding of sentences using tokenize of openai/whisper-small
# processor for marathi language is selected since we deal with marathi audio files

from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="marathi", task="transcribe"
)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

In [ ]:
#Sampling rate of openai/whisper small processor is 16000 Hz so we need to re sample our audio column to match it
# Cast column is used to resample our audio signals

from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
# downsampled audio dataset
# now we can prepare datset for training
common_voice["train"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None)}

In [ ]:
# fuction which converts audio samplesto log-mel spectograms using feature extractorof processor
# labels the text using tokenizor of processor
# this function returns daatset exaample containing new columns [input_features,labels,input_length] along with other old colums
def prepare_dataset(example):
    audio = example["audio"]

    example = processor(
        audio=audio["array"],
        sampling_rate=audio["sampling_rate"],
        text=example["sentence"],
    )

    # compute input length of audio sample in seconds
    example["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    return example

In [ ]:
# mapping common_voice dataset and removing old input columns audio and sentence

common_voice = common_voice.map(
    prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1
)

Map:   0%|          | 0/4008 [00:00<?, ? examples/s]

Map:   0%|          | 0/1737 [00:00<?, ? examples/s]

In [ ]:
common_voice["train"].features

{'input_features': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_length': Value(dtype='float64', id=None)}

In [ ]:
max_input_length = 30.0

# function to return samples whose audio length is of 30 sec which is fixed to avoid additionsl padding in next steps
def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
# filtering audio samples with less than equal to 30 seconds
common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/4008 [00:00<?, ? examples/s]

In [ ]:
# filtering audio samples with less than equal to 30 seconds
common_voice["train"] = common_voice["train"].filter(
    is_audio_in_length_range,
    input_columns=["input_length"],
)

Filter:   0%|          | 0/4008 [00:00<?, ? examples/s]

In [ ]:
common_voice["train"].features

{'input_features': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_length': Value(dtype='float64', id=None)}

In [ ]:
# encoded label id's created for text tokens for each sentences encoded by tokenizer of openai/whisper-small
common_voice["train"][0]['labels']

[50258,
 50320,
 50359,
 50363,
 3941,
 228,
 3941,
 230,
 3941,
 248,
 21981,
 8485,
 228,
 3941,
 250,
 17937,
 25411,
 3941,
 103,
 3941,
 96,
 8485,
 113,
 17937,
 3941,
 95,
 36158,
 8485,
 248,
 17937,
 46758,
 46758,
 21981,
 11,
 8485,
 97,
 45938,
 36158,
 3941,
 114,
 31881,
 48449,
 3941,
 98,
 31881,
 44500,
 31881,
 8485,
 101,
 31881,
 3941,
 253,
 8485,
 244,
 17937,
 36158,
 3941,
 103,
 31881,
 36158,
 35082,
 17937,
 3941,
 114,
 31881,
 8485,
 251,
 17937,
 46758,
 31881,
 13,
 50257]

In [ ]:
# defining collator function to returns tensor pt
# collator will be provided to Trainer() as a argument in further steps
# padding will happen with tokenizer pad method and feature extractor pad method
# since input feature are of fixed length which we already filtered no additional padding required for input_features
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [
            {"input_features": feature["input_features"][0]} for feature in features
        ]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
# calling Datacollator ro returns tensor pt's

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.5 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
# imprting normalizer of text to remove punctuations from text

from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

# function which returns wer by comparing normalised predicted text and normalised referenced text for evaluation

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer_ortho = 100 * metric.compute(predictions=pred_str, references=label_str)

    # compute normalised WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]
    # filtering step to only evaluate the samples that correspond to non-zero references:
    pred_str_norm = [
        pred_str_norm[i] for i in range(len(pred_str_norm)) if len(label_str_norm[i]) > 0
    ]
    label_str_norm = [
        label_str_norm[i]
        for i in range(len(label_str_norm))
        if len(label_str_norm[i]) > 0
    ]

    wer = 100 * metric.compute(predictions=pred_str_norm, references=label_str_norm)

    return {"wer_ortho": wer_ortho, "wer": wer}

In [ ]:
# pre trained condional chekpoint before training

from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(
    model.generate, language="marathi", task="transcribe", use_cache=True
)

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate

accelerate.__version__

'0.25.0'

In [ ]:
from transformers import TrainingArguments

In [ ]:
# creating repo to store model in hugging face

from huggingface_hub import create_repo
create_repo("whisper-small-mr", private=False)


In [ ]:
# hyper parameter tuning for training our model
# check documentation for reasons behind parameter selction

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-mr",  # name on the HF Hub
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    lr_scheduler_type="constant_with_warmup",
    warmup_steps=50,
    max_steps=500,  # increase to 4000 if you have your own GPU or a Colab paid plan
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
common_voice["test"].features

{'input_features': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_length': Value(dtype='float64', id=None)}

In [ ]:
common_voice["train"].features

{'input_features': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'input_length': Value(dtype='float64', id=None)}

In [ ]:
# passing all training arguments, collator,compute metrics, tokenizer, train and test dataset along with model for sequential training

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [ ]:
# loading train method for training our jagadeeshjagat/whisper-small-mr ASR model
trainer.train()



/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer Ortho,Wer
500,0.192000,0.267446,47.420808,18.864092


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=500, training_loss=0.37613897514343264, metrics={'train_runtime': 4462.4924, 'train_samples_per_second': 1.793, 'train_steps_per_second': 0.112, 'total_flos': 2.30637451935744e+18, 'train_loss': 0.37613897514343264, 'epoch': 1.99})

**Results :**

Got WER of 18 which is good within training period of 1.5 hrs of audio data.
we can improve it by increasing evaluation steps from 500 to 2000 and above.
Due to memory concerns not moving further, since it is also good wer.

In [ ]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_14_0",
    "dataset": "Common Voice 14 mr",  # a 'pretty' name for the training dataset
    "language": "mr",
    "model_name": "Whisper Small Mr - Jagadeesh",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [ ]:
# pushing my pre trained model to my hugging face hub repocreate earlier to load it in future
trainer.push_to_hub(**kwargs)

'https://huggingface.co/jagadeeshjagat/whisper-small-mr/tree/main/'